In [489]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mp
import matplotlib.pyplot as plt
import glob
import os
import datetime
import string
import logging
import sqlite3
from datetime import datetime, date 
import ast
import numpy as np
from sqlalchemy import create_engine
import logging
import unittest



In [490]:
os.chdir('/Users/njaro/Downloads/subscriber-pipeline-starter-kit/subscriber-pipeline-starter-kit/dev')

In [491]:
con = sqlite3.connect('Cademycode.db')
cur = con.cursor()

In [492]:
table_list = [a for a in cur.execute("select name from sqlite_master where type == 'table'")]
print(table_list)

[('cademycode_students',), ('cademycode_courses',), ('cademycode_student_jobs',)]


In [493]:
students = pd.read_sql_query('select * from cademycode_students', con)
courses = pd.read_sql_query('select * from cademycode_courses', con)
student_job = pd.read_sql_query('select * from cademycode_student_jobs', con)

In [494]:
print('students', len(students))
print('courses', len(courses))
print('student_job', len(student_job))

students 5000
courses 10
student_job 13


In [495]:
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB


In [496]:
type(students['contact_info'][0])

str

In [497]:
students['contact_info'] = students['contact_info'].apply(lambda x: ast.literal_eval(x))
new_contact = pd.json_normalize(students['contact_info'])
students = pd.concat([students.drop('contact_info', axis=1), new_contact], axis=1)
students.head()

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com


In [498]:
split_address = students.mailing_address.str.split(',', expand=True)
split_address.columns = ['street address', 'city', 'state', 'zip code']
students =  pd.concat([students.drop('mailing_address', axis = 1), split_address], axis=1)
students.head()

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street address,city,state,zip code
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,rubio6772@hmail.com,767 Crescent Fair,Shoals,Indiana,37439
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,hosea_dale8084@coldmail.com,P.O. Box 41269,St. Bonaventure,Virginia,83637
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,kirk4005@hmail.com,517 SE Wintergreen Isle,Lane,Arkansas,82242
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,alexander9810@hmail.com,18 Cinder Cliff,Doyles borough,Rhode Island,73737


In [499]:
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   job_id                  4995 non-null   object
 5   num_course_taken        4749 non-null   object
 6   current_career_path_id  4529 non-null   object
 7   time_spent_hrs          4529 non-null   object
 8   email                   5000 non-null   object
 9   street address          5000 non-null   object
 10  city                    5000 non-null   object
 11  state                   5000 non-null   object
 12  zip code                5000 non-null   object
dtypes: int64(1), object(12)
memory usage: 507.9+ KB


In [500]:
students_dob = students['dob']
students_dob_df = pd.DataFrame(data = students_dob)
def age(born):
    
    
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today() 
    return today.year - born.year - ((today.month,  
                                      today.day) < (born.month,  
                                                    born.day)) 
students_dob_df['Age'] = students_dob_df['dob'].apply(age) 


In [501]:
update_students = pd.concat([students, students_dob_df], axis=1)


In [502]:
update_students.describe()

,uuid,Age
count,5000.000000,5000.000000
mean,2500.500000,51.120400
std,1443.520003,18.325247
min,1.000000,19.000000
25%,1250.750000,35.000000
50%,2500.500000,51.000000
75%,3750.250000,67.000000
max,5000.000000,82.000000


In [503]:
students.value_counts()

uuid  name                    dob         sex  job_id  num_course_taken  current_career_path_id  time_spent_hrs  email                           street address       city               state            zip code
1     Annabelle Avery         1943-07-03  F    7.0     6.0               1.0                     4.99            annabelle_avery9376@woohoo.com  303 N Timber Key      Irondale           Wisconsin        84736      1
3337  Greg Allan              1968-09-16  M    5.0     1.0               7.0                     24.78           allan8306@inlook.com            554 Broad Rose        Flagler Beach      Florida          65799      1
3322  Jerald Mendez           1961-09-18  M    2.0     4.0               10.0                    6.44            jerald_mendez392@hmail.com      700 SE Quail Vista    Petrey             New Mexico       49882      1
3323  Clarisa Oneill          1953-12-21  F    4.0     10.0              8.0                     16.49           clarisa_oneill7269@inlook.co

In [504]:
update_students.head()

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street address,city,state,zip code,dob,Age
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736,1943-07-03,81
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,rubio6772@hmail.com,767 Crescent Fair,Shoals,Indiana,37439,1991-02-07,33
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,hosea_dale8084@coldmail.com,P.O. Box 41269,St. Bonaventure,Virginia,83637,1989-12-07,34
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,kirk4005@hmail.com,517 SE Wintergreen Isle,Lane,Arkansas,82242,1988-07-31,36
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,alexander9810@hmail.com,18 Cinder Cliff,Doyles borough,Rhode Island,73737,1963-08-31,61


In [505]:
update_students['job_id'] = update_students['job_id'].astype(float)
update_students['current_career_path_id'] = update_students['current_career_path_id'].astype(float)
update_students['num_course_taken'] = update_students['num_course_taken'].astype(float)
update_students['time_spent_hrs'] = update_students['time_spent_hrs'].astype(float)
update_students['Age'] = update_students['Age'].astype(float)
type(update_students['Age'][0])

numpy.float64

In [506]:
missing_course_taken = students[students[['num_course_taken']].isnull().any(axis=1)]
missing_data = pd.DataFrame()
missing_data = pd.concat([missing_data, missing_course_taken])
update_students = update_students.dropna(subset=['num_course_taken'])
update_students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4749 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uuid                    4749 non-null   int64  
 1   name                    4749 non-null   object 
 2   dob                     4749 non-null   object 
 3   sex                     4749 non-null   object 
 4   job_id                  4744 non-null   float64
 5   num_course_taken        4749 non-null   float64
 6   current_career_path_id  4298 non-null   float64
 7   time_spent_hrs          4298 non-null   float64
 8   email                   4749 non-null   object 
 9   street address          4749 non-null   object 
 10  city                    4749 non-null   object 
 11  state                   4749 non-null   object 
 12  zip code                4749 non-null   object 
 13  dob                     4749 non-null   object 
 14  Age                     4749 non-null   float

In [507]:
missing_job_id = update_students[update_students[['job_id']].isnull().any(axis=1)]
missing_job_id.head()

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street address,city,state,zip code,dob,Age
162,163,Glen Riley,2002-08-22,M,NaN,8.0,3.0,5.70,glen_riley4484@hmail.com,P.O. Box 37267,Cornlea village,Tennessee,19192,2002-08-22,22.0
757,758,Mercedez Vorberg,2002-03-25,F,NaN,15.0,4.0,4.14,mercedez6297@woohoo.com,284 Cedar Seventh,Virden village,Washington,60489,2002-03-25,22.0
854,855,Kurt Ho,2002-05-29,M,NaN,0.0,8.0,23.72,ho6107@inlook.com,P.O. Box 27254,Olin,New Hampshire,60067,2002-05-29,22.0
1029,1030,Penny Gaines,2002-03-01,N,NaN,15.0,4.0,16.25,gaines2897@hmail.com,138 Misty Vale,Stockton borough,West Virginia,53630,2002-03-01,22.0
1542,1543,Frederick Reilly,2002-11-13,M,NaN,7.0,9.0,21.32,frederick_reilly6971@woohoo.com,P.O. Box 40769,Quakervillage,Maryland,96218,2002-11-13,21.0


In [508]:

update_students = update_students.dropna(subset=['job_id'])
update_students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uuid                    4744 non-null   int64  
 1   name                    4744 non-null   object 
 2   dob                     4744 non-null   object 
 3   sex                     4744 non-null   object 
 4   job_id                  4744 non-null   float64
 5   num_course_taken        4744 non-null   float64
 6   current_career_path_id  4293 non-null   float64
 7   time_spent_hrs          4293 non-null   float64
 8   email                   4744 non-null   object 
 9   street address          4744 non-null   object 
 10  city                    4744 non-null   object 
 11  state                   4744 non-null   object 
 12  zip code                4744 non-null   object 
 13  dob                     4744 non-null   object 
 14  Age                     4744 non-null   float

In [509]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes


In [510]:
courses.describe()

,career_path_id,hours_to_complete
count,10.00000,10.000000
mean,5.50000,21.900000
std,3.02765,6.707376
min,1.00000,12.000000
25%,3.25000,18.500000
50%,5.50000,20.000000
75%,7.75000,26.500000
max,10.00000,35.000000


In [511]:
courses.value_counts()

career_path_id  career_path_name           hours_to_complete
1               data scientist             20                   1
2               data engineer              20                   1
3               data analyst               12                   1
4               software engineering       25                   1
5               backend engineer           18                   1
6               frontend engineer          20                   1
7               iOS developer              27                   1
8               android developer          27                   1
9               machine learning engineer  35                   1
10              ux/ui designer             15                   1
Name: count, dtype: int64

In [512]:
courses.head()

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18


In [513]:
courses[courses.isnull().any(axis=1)]

,career_path_id,career_path_name,hours_to_complete


In [514]:
student_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        13 non-null     int64 
 1   job_category  13 non-null     object
 2   avg_salary    13 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 444.0+ bytes


In [515]:
student_job.head()

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000


In [516]:
student_job.value_counts()

job_id  job_category        avg_salary
3       software developer  110000        2
4       creative            66000         2
5       financial services  135000        2
0       other               80000         1
1       analytics           86000         1
2       engineer            101000        1
6       education           61000         1
7       HR                  80000         1
8       student             10000         1
9       healthcare          120000        1
Name: count, dtype: int64

In [517]:
student_job.describe()

,job_id,avg_salary
count,13.000000,13.000000
mean,4.384615,89230.769231
std,2.662657,34727.879881
min,0.000000,10000.000000
25%,3.000000,66000.000000
50%,4.000000,86000.000000
75%,6.000000,110000.000000
max,9.000000,135000.000000


In [518]:
student_job[student_job.isnull().any(axis=1)]

,job_id,job_category,avg_salary


In [519]:
update_students.to_csv(r'C:\Users\njaro\Downloads\update_students.csv')

In [520]:
student_job.to_csv(r'C:\Users\njaro\Downloads\student_job.csv')

In [521]:
courses.to_csv(r'C:\Users\njaro\Downloads\Courses.csv')

In [522]:
update_students  = r'C:\Users\njaro\Downloads\update_students.csv'
update_students_db = pd.read_csv(update_students)
student_job = r'C:\Users\njaro\Downloads\student_job.csv'
student_job_db = pd.read_csv(student_job)
courses = r'C:\Users\njaro\Downloads\Courses.csv'
courses_db = pd.read_csv(courses)

In [523]:
import sqlite3

def get_schema(update_students_db):
    # Connect to the database
    con = sqlite3.connect(update_students_db)
    curs = con.cursor()
    
    # Get the schema information
    curs.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    schema = curs.fetchall()
    
    # Close the connection
    con.close()
    
    return schema

# Paths to the original and updated databases
original_update_students = 'update_students_db'
updated_db_path = 'updated_database_db'

# Get schemas
original_schema = get_schema('update_students_db')
updated_schema = get_schema('updated_students_db')

# Compare schemas
if original_schema == updated_schema:
    print("The schemas are the same.")
else:
    print("The schemas are different.")
    print("Original Schema:")
    for table in original_schema:
        print(table[0])
    print("\nUpdated Schema:")
    for table in updated_schema:
        print(table[0])


The schemas are the same.


In [524]:
def get_schema(student_job_db):
    # Connect to the database
    con = sqlite3.connect(student_job_db)
    curs = con.cursor()
    
    # Get the schema information
    curs.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    schema = curs.fetchall()
    
    # Close the connection
    con.close()
    
    return schema

# Paths to the original and updated databases
original_update_students = 'student_job_db'
updated_db_path = 'updated_student_job'

# Get schemas
original_schema = get_schema('student_job_db')
updated_schema = get_schema('updated_students__job_db')

# Compare schemas
if original_schema == updated_schema:
    print("The schemas are the same.")
else:
    print("The schemas are different.")
    print("Original Schema:")
    for table in original_schema:
        print(table[0])
    print("\nUpdated Schema:")
    for table in updated_schema:
        print(table[0])


The schemas are the same.


In [525]:
def get_schema(courses_db):
    # Connect to the database
    con = sqlite3.connect(courses_db)
    curs = con.cursor()
    
    # Get the schema information
    curs.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    schema = curs.fetchall()
    
    # Close the connection
    con.close()
    
    return schema

# Paths to the original and updated databases
original_courses_db = 'courses_db'
updated_db_path = 'updated_courses'

# Get schemas
original_schema = get_schema('courses_db')
updated_schema = get_schema('updated_courses_db')

# Compare schemas
if original_schema == updated_schema:
    print("The schemas are the same.")
else:
    print("The schemas are different.")
    print("Original Schema:")
    for table in original_schema:
        print(table[0])
    print("\nUpdated Schema:")
    for table in updated_schema:
        print(table[0])


The schemas are the same.
